In [ ]:
import pymysql
import pandas as pd

In [ ]:
_db = pymysql.connect(
    user = 'root', 
    password = '1234', 
    host = 'localhost', 
    db = 'ubion', 
    port = 3306
)

In [ ]:
cursor = _db.cursor(pymysql.cursors.DictCursor)

In [ ]:
sql = """
        select 
        * 
        from 
        emp 
"""
cursor.execute(sql)
result = cursor.fetchall()
pd.DataFrame(result)

### 부서 이름이 'sales'인 소속 사원들 출력
1. dept 테이블에서 부서이름이 sales 인 deptno을 가지고 오는 작업
2. emp 테이블에서 1번 과정에서 나온 deptno를 기준으로 사원 정보 출력

In [ ]:
sql = """
    select 
    DEPTNO 
    from 
    dept 
    where 
    DNAME = 'sales'
"""

cursor.execute(sql)
result = cursor.fetchall()
print(result)

In [ ]:
# sql = """
#         select 
#         * 
#         from 
#         emp 
#         where 
#         DEPTNO = 30
# """

sql = """
        select 
        * 
        from 
        emp 
        where 
        DEPTNO = (
                select 
                DEPTNO 
                from 
                dept 
                where 
                DNAME = 'sales'
        )
"""

cursor.execute(sql)
result = cursor.fetchall()
pd.DataFrame(result)

In [ ]:
## 부서 번호가 10 아니면 30인 경우의 사원의 정보를 출력하시오

#case1
sql = """
        select 
        * 
        from 
        emp 
        where 
        DEPTNO = 10.0 or DEPTNO = 30.0
"""

#case2
sql2 = """
        select 
        * 
        from 
        emp 
        where 
        DEPTNO 
        in 
        (10, 30)
"""
cursor.execute(sql)
result = cursor.fetchall()
cursor.execute(sql2)
result2 = cursor.fetchall()
print(pd.DataFrame(result))
print(pd.DataFrame(result2))

In [ ]:
## SAL 2000 이상이고 5000 이하인 사원 정보를 출력
#case1
sql = """
        select 
        * 
        from 
        emp 
        where
        SAL >= 2000 
        and 
        SAL <= 5000
"""
#case2
sql2 = """
        select 
        * 
        from
        emp 
        where 
        SAL 
        between
        2000 and 5000
"""
cursor.execute(sql)
result = cursor.fetchall()
cursor.execute(sql2)
result2 = cursor.fetchall()
print(pd.DataFrame(result))
print(pd.DataFrame(result2))


### 파생변수 생성
1. job_code / job 컬럼의 값이 salesman -> 1, manager -> 2, 그 외의 직업은 3
2. EMPNO, ENAME, job_code 출력

In [ ]:
sql = """
    select 
    EMPNO, 
    ENAME, 
    JOB, 
    case
        when JOB = 'SALESMAN'
        then '1'
        when JOB = 'MANAGER' 
        then '2'
        else '3'
    end as job_code 
    from 
    emp
"""
cursor.execute(sql)
result = cursor.fetchall()
pd.DataFrame(result)

In [ ]:
sql = """
    select * from emp
"""
cursor.execute(sql)
result = cursor.fetchall()
df = pd.DataFrame(result)
df

### 위에서 sql로 작업한 파생변수 만들기를 pandas에서 다시 한번 출력

In [ ]:
## apply함수를 이용하여 파생변수 생성 

## lambda 함수 사용
df2 = df.copy()
df2["job_code"] = df2["JOB"].apply(lambda x : "1" if (x == "SALESMAN") else \
    ("2" if (x == "MANAGER") else "3"))
df2[["EMPNO", "ENAME", "JOB", "job_code"]]


In [ ]:
## 일반 함수 사용
def change(x):
    if x == "SALESMAN":
        return "1"
    elif x == "MANAGER":
        return "2"
    else:
        return "3"
df3 = df.copy()
df3["job_code"] = df3["JOB"].apply(change)
df3[["EMPNO", "ENAME", "JOB", "job_code"]]

In [35]:
## 글자를 대체하는 방법 
sql = """
    select 
    JOB, 
    replace(JOB, "SALESMAN", "SALES") as JOB_
    from 
    emp
"""
cursor.execute(sql)
result = cursor.fetchall()
pd.DataFrame(result).head(3)

,JOB,JOB_
0,CLERK,CLERK
1,SALESMAN,SALES
2,SALESMAN,SALES


In [ ]:
## pandas에서 위의 sql 출력과 같이 출력
# 파생변수 JOB_
df4 = df.copy()
df4["JOB_"] = df4["JOB"].str.replace("SALESMAN", "SALES")
df4[["JOB", "JOB_"]]

In [ ]:
## 컬럼 2개의 값을 하나의 컬럼에 표시
sql = """
    select 
    concat(ENAME, ' (', JOB, ')') as ENAME_ 
    from emp
"""
cursor.execute(sql)
result = cursor.fetchall()
pd.DataFrame(result)

In [42]:
import mod_sql as m_s

In [ ]:
df5 = df.copy()
df5["ENAME_"] = df5["ENAME"] + "(" + df5["JOB"] + ")"
df5[["ENAME_"]]

In [62]:
## 모듈 클래스 생성
db = m_s.Database()

In [ ]:
## sql에서 if문을 사용하여 새로운 파생변수 생성
## SAL 2000 이상이면 High 미만이면 Low 
## 컬럼의 이름은 H/L
sql = """
        select 
        SAL, 
        if(SAL >= 2000, "High", "Low") as `H/L` 
        from 
        emp
"""
db.executeall(sql)

In [ ]:
df6 = df.copy()
df6["H/L"] = df6["SAL"].apply(lambda x : "High" if x >=2000 else "Low")
df6[["SAL", "H/L"]]

In [47]:
def change(x):
    if x >= 2000:
        return "High"
    else:
        return "Low"

df7 = df.copy()
df7["H/L"] = df7["SAL"].apply(change)
df7[["SAL", "H/L"]]

,SAL,H/L
0,800.0,Low
1,1600.0,Low
2,1250.0,Low
3,2975.0,High
4,1250.0,Low
5,2850.0,High
6,2450.0,High
7,3000.0,High
8,5000.0,High
9,1500.0,Low


In [54]:
## join 
sql = """
        select 
        * 
        from 
        emp 
        left join 
        dept 
        on 
        emp.DEPTNO = dept.DEPTNO
"""
db.executeall(sql)

,EMPNO,ENAME,JOB,MGR,HIREDATE,SAL,COMM,DEPTNO,dept.DEPTNO,DNAME,LOC
0,7369,SMITH,CLERK,7902.0,1980-12-17,800.0,0.0,20.0,20.0,RESEARCH,DALLAS
1,7499,ALLEN,SALESMAN,7698.0,1981-02-20,1600.0,300.0,30.0,30.0,SALES,CHICAGO
2,7521,WARD,SALESMAN,7698.0,1981-02-22,1250.0,500.0,30.0,30.0,SALES,CHICAGO
3,7566,JONES,MANAGER,7839.0,1981-04-02,2975.0,0.0,20.0,20.0,RESEARCH,DALLAS
4,7654,MARTIN,SALESMAN,7698.0,1981-09-28,1250.0,1400.0,30.0,30.0,SALES,CHICAGO
5,7698,BLAKE,MANAGER,7839.0,1981-05-01,2850.0,0.0,30.0,30.0,SALES,CHICAGO
6,7782,CLARK,MANAGER,7839.0,1981-06-09,2450.0,0.0,10.0,10.0,ACCOUNTING,NEW YORK
7,7788,SCOTT,ANALYST,7566.0,1987-06-28,3000.0,0.0,20.0,20.0,RESEARCH,DALLAS
8,7839,KING,PRESIDENT,0.0,1981-11-17,5000.0,0.0,10.0,10.0,ACCOUNTING,NEW YORK
9,7844,TURNER,SALESMAN,7698.0,1981-09-08,1500.0,0.0,30.0,30.0,SALES,CHICAGO


In [55]:
sql = """
        select 
        * 
        from 
        emp 
        right join 
        dept 
        on
        emp.DEPTNO = dept.DEPTNO
"""
db.executeall(sql)

,EMPNO,ENAME,JOB,MGR,HIREDATE,SAL,COMM,DEPTNO,dept.DEPTNO,DNAME,LOC
0,7934.0,MILLER,CLERK,7782.0,1982-01-23,1300.0,0.0,10.0,10,ACCOUNTING,NEW YORK
1,7839.0,KING,PRESIDENT,0.0,1981-11-17,5000.0,0.0,10.0,10,ACCOUNTING,NEW YORK
2,7782.0,CLARK,MANAGER,7839.0,1981-06-09,2450.0,0.0,10.0,10,ACCOUNTING,NEW YORK
3,7902.0,FORD,ANALYST,7566.0,1981-12-03,3000.0,0.0,20.0,20,RESEARCH,DALLAS
4,7876.0,ADAMS,CLERK,7788.0,1987-07-13,1100.0,0.0,20.0,20,RESEARCH,DALLAS
5,7788.0,SCOTT,ANALYST,7566.0,1987-06-28,3000.0,0.0,20.0,20,RESEARCH,DALLAS
6,7566.0,JONES,MANAGER,7839.0,1981-04-02,2975.0,0.0,20.0,20,RESEARCH,DALLAS
7,7369.0,SMITH,CLERK,7902.0,1980-12-17,800.0,0.0,20.0,20,RESEARCH,DALLAS
8,7900.0,JAMES,CLERK,7698.0,1981-12-03,950.0,0.0,30.0,30,SALES,CHICAGO
9,7844.0,TURNER,SALESMAN,7698.0,1981-09-08,1500.0,0.0,30.0,30,SALES,CHICAGO


In [65]:
sql = """
        insert into 
        emp 
        values 
        (%s, %s, %s, %s, %s, %s, %s, %s) 
"""
values = [8000, "test", "CEO", 0, "1988-01-01", 3500.0, 100, 50]
db.execute(sql, values)
db.commit()

In [56]:
## inner join 
sql = """
        select 
        * 
        from 
        emp 
        inner join 
        dept 
        on 
        emp.DEPTNO = dept.DEPTNO
"""
db.executeall(sql)

,EMPNO,ENAME,JOB,MGR,HIREDATE,SAL,COMM,DEPTNO,dept.DEPTNO,DNAME,LOC
0,7369,SMITH,CLERK,7902.0,1980-12-17,800.0,0.0,20.0,20,RESEARCH,DALLAS
1,7499,ALLEN,SALESMAN,7698.0,1981-02-20,1600.0,300.0,30.0,30,SALES,CHICAGO
2,7521,WARD,SALESMAN,7698.0,1981-02-22,1250.0,500.0,30.0,30,SALES,CHICAGO
3,7566,JONES,MANAGER,7839.0,1981-04-02,2975.0,0.0,20.0,20,RESEARCH,DALLAS
4,7654,MARTIN,SALESMAN,7698.0,1981-09-28,1250.0,1400.0,30.0,30,SALES,CHICAGO
5,7698,BLAKE,MANAGER,7839.0,1981-05-01,2850.0,0.0,30.0,30,SALES,CHICAGO
6,7782,CLARK,MANAGER,7839.0,1981-06-09,2450.0,0.0,10.0,10,ACCOUNTING,NEW YORK
7,7788,SCOTT,ANALYST,7566.0,1987-06-28,3000.0,0.0,20.0,20,RESEARCH,DALLAS
8,7839,KING,PRESIDENT,0.0,1981-11-17,5000.0,0.0,10.0,10,ACCOUNTING,NEW YORK
9,7844,TURNER,SALESMAN,7698.0,1981-09-08,1500.0,0.0,30.0,30,SALES,CHICAGO


1. emp table, dept table을 sql을 이용하여 DataFrame 생성
2. merge()함수를 이용해서 left, right, inner join을 실행

In [ ]:
sql = "select * from emp"
sql2 = "select * from dept"
emp = db.executeall(sql)
dept = db.executeall(sql2)
print(emp)
print(dept)

In [ ]:
pd.merge(emp, dept, on="DEPTNO", how="left")

In [ ]:
pd.merge(emp, dept, on="DEPTNO", how="right")

In [ ]:
pd.merge(emp, dept, on="DEPTNO", how="inner")

In [ ]:
pd.merge(emp, dept, on="DEPTNO", how="outer")

### 이중 쿼리문
1. 부서 위치가 'NEW YORK', 'DALLAS'가 아닌 소속 사원의 정보 출력
2. 부서의 위치가 'NEW YORK', 'DALLAS'가 아닌 부서 번호 출력
3. 사원 정보 테이블에서 위에서 찾은 부서 번호를 조건으로 사원 정보를 출력

In [76]:
## 2번 과정
sql = """
        select 
        * 
        from 
        dept 
        where 
        LOC 
        not in 
        ('NEW YORK', 'DALLAS') 
"""
cursor.execute(sql)
result = cursor.fetchall()
print(result)

[{'DEPTNO': 30, 'DNAME': 'SALES', 'LOC': 'CHICAGO'}, {'DEPTNO': 40, 'DNAME': 'OPERATIONS', 'LOC': 'BOSTON'}]


In [75]:
sql = """
        select 
        * 
        from 
        emp 
        where 
        DEPTNO 
        in 
        (
            select DEPTNO from dept where LOC not in ('NEW YORK', 'DALLAS')
        )
"""
db.executeall(sql)

OperationalError: (1241, 'Operand should contain 1 column(s)')